In [ ]:
!kaggle datasets download -d agrigorev/clothing-dataset-full

In [ ]:
!unzip clothing-dataset-full.zip

In [ ]:
import pathlib
import shutil
import os
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
import numpy as np
import json
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("images.csv")

# Drop few clasess for simplicity
mask = df["label"].isin(["Not sure", "Other", "Blouse", "Top", "Skip"])
df = df.drop(df[mask].index)
print(df.groupby("label").count())

labels = sorted(df["label"].unique())
labels = {id_: label for id_, label in enumerate(labels)}
with open("labels.json", "w") as file:
    json.dump(labels, file)
images = [(f"{name}.jpg", label) for name, label in zip(df["image"], df["label"])]

In [ ]:
old_folder = pathlib.Path("images_compressed")
new_folder = pathlib.Path("images_for_model")
for name, label in images:
    target_folder = new_folder / label
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    shutil.copy(src=old_folder / name, dst=target_folder / name)

In [ ]:
train_dataset = keras.utils.image_dataset_from_directory(
    new_folder,
    batch_size=32,
    image_size=(180, 180),
    validation_split=0.2,
    subset="training",
    seed=1,
    label_mode="categorical"
)

val_dataset = keras.utils.image_dataset_from_directory(
    new_folder,
    batch_size=32,
    image_size=(180, 180),
    validation_split=0.2,
    subset="validation",
    seed=1,
    label_mode="categorical"
)

In [ ]:
conv_base = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False)
conv_base.trainable = False

data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomZoom(0.2),
        keras.layers.RandomRotation(0.1)
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(15, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="clothes_classifier_vgg16_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    train_dataset,
    batch_size=32,
    epochs=15,
    validation_data=val_dataset,
    callbacks=callbacks)

In [ ]:
model = keras.models.load_model("clothes_classifier_vgg16_with_augmentation.keras")

vgg16 = model.get_layer("vgg16")
vgg16.get_layer("block5_conv1").trainable = True
vgg16.get_layer("block5_conv2").trainable = True
vgg16.get_layer("block5_conv3").trainable = True

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="clothes_classifier_vgg16_with_augmentation_and_fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    train_dataset,
    batch_size=32,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks)

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.show()